In [143]:
import kagglehub
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import random
import os

# Download latest version
# path = kagglehub.dataset_download("utsavdey1410/food-nutrition-dataset")

# print("Path to dataset files:", path)

In [144]:


# Directory containing the dataset files
data_dir = "FINAL FOOD DATASET/DATASET"

# List to store each DataFrame
df_list = []

# Loop through each file in the directory
for filename in os.listdir(data_dir):
    if filename.endswith(".csv"):
        file_path = os.path.join(data_dir, filename)
        # Read each CSV file and append to the list
        df = pd.read_csv(file_path, usecols=lambda column: "Unnamed" not in column) 
        df_list.append(df)

# Concatenate all DataFrames in the list into a single DataFrame
combined_df = pd.concat(df_list, ignore_index=True)

# Display the first few rows of the combined DataFrame
combined_df.head()


,food,Caloric Value,Fat,Saturated Fats,Monounsaturated Fats,Polyunsaturated Fats,Carbohydrates,Sugars,Protein,Dietary Fiber,...,Calcium,Copper,Iron,Magnesium,Manganese,Phosphorus,Potassium,Selenium,Zinc,Nutrition Density
0,cream cheese,51.0,5.0,2.9,1.3,0.200,0.8,0.500,0.9,0.0,...,0.008,14.100,0.082,0.027,1.300,0.091,15.5,19.100,0.039,7.070
1,neufchatel cheese,215.0,19.4,10.9,4.9,0.800,3.1,2.700,7.8,0.0,...,99.500,0.034,0.100,8.500,0.088,117.300,129.2,0.054,0.700,130.100
2,requeijao cremoso light catupiry,49.0,3.6,2.3,0.9,0.000,0.9,3.400,0.8,0.1,...,0.000,0.000,0.000,0.000,0.000,0.000,0.0,0.000,0.000,5.400
3,ricotta cheese,30.0,2.0,1.3,0.5,0.002,1.5,0.091,1.5,0.0,...,0.097,41.200,0.097,0.096,4.000,0.024,30.8,43.800,0.035,5.196
4,cream cheese low fat,30.0,2.3,1.4,0.6,0.042,1.2,0.900,1.2,0.0,...,22.200,0.072,0.008,1.200,0.098,22.800,37.1,0.034,0.053,27.007


In [145]:
combined_df.columns

Index(['food', 'Caloric Value', 'Fat', 'Saturated Fats',
       'Monounsaturated Fats', 'Polyunsaturated Fats', 'Carbohydrates',
       'Sugars', 'Protein', 'Dietary Fiber', 'Cholesterol', 'Sodium', 'Water',
       'Vitamin A', 'Vitamin B1', 'Vitamin B11', 'Vitamin B12', 'Vitamin B2',
       'Vitamin B3', 'Vitamin B5', 'Vitamin B6', 'Vitamin C', 'Vitamin D',
       'Vitamin E', 'Vitamin K', 'Calcium', 'Copper', 'Iron', 'Magnesium',
       'Manganese', 'Phosphorus', 'Potassium', 'Selenium', 'Zinc',
       'Nutrition Density'],
      dtype='object')

In [146]:
combined_df.describe()

,Caloric Value,Fat,Saturated Fats,Monounsaturated Fats,Polyunsaturated Fats,Carbohydrates,Sugars,Protein,Dietary Fiber,Cholesterol,...,Calcium,Copper,Iron,Magnesium,Manganese,Phosphorus,Potassium,Selenium,Zinc,Nutrition Density
count,2435.000000,2435.000000,2435.000000,2435.000000,2435.000000,2435.000000,2435.000000,2435.000000,2435.000000,2435.000000,...,2435.000000,2435.000000,2435.000000,2435.000000,2435.000000,2435.000000,2435.000000,2435.000000,2435.000000,2435.000000
mean,228.817123,11.540861,5.461358,5.325789,3.315416,19.845633,5.757806,14.665413,3.299560,67.995004,...,54.719559,11.877601,2.917156,36.541743,6.729074,160.859170,312.326765,56.274167,2.654360,110.858098
std,407.437925,38.687985,34.149622,27.106086,24.583924,37.902969,27.767940,40.927214,23.771054,438.366022,...,125.067739,83.907175,23.773927,80.382323,32.896669,353.273309,638.186946,216.117818,23.817869,195.119334
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,45.000000,0.300000,0.064000,0.058000,0.071000,0.500000,0.000000,0.800000,0.000000,0.000000,...,0.600000,0.041000,0.100000,1.550000,0.038000,0.800000,27.800000,0.016000,0.055000,16.951500
50%,117.000000,2.200000,0.500000,0.500000,0.400000,7.000000,0.091000,3.500000,0.300000,0.000000,...,14.200000,0.100000,0.600000,10.800000,0.200000,43.000000,113.600000,0.053000,0.300000,54.300000
75%,261.500000,9.600000,2.700000,3.550000,1.800000,25.300000,3.300000,13.600000,2.300000,27.050000,...,49.450000,0.500000,1.900000,37.450000,0.800000,175.950000,347.600000,0.091000,1.200000,135.926500
max,6077.000000,722.000000,722.000000,722.000000,722.000000,722.000000,722.000000,722.000000,722.000000,10509.000000,...,1283.500000,1890.000000,722.000000,921.600000,722.000000,5490.000000,11336.900000,3308.000000,722.000000,3911.400000


In [147]:
per100g_cols = [
    'Caloric Value', 'Fat', 'Saturated Fats', 'Monounsaturated Fats', 'Polyunsaturated Fats', 
    'Carbohydrates', 'Sugars', 'Protein', 'Dietary Fiber', 'Cholesterol', 'Sodium', 'Water',
    'Vitamin A', 'Vitamin B1', 'Vitamin B11', 'Vitamin B12', 'Vitamin B2', 'Vitamin B3', 'Vitamin B5', 
    'Vitamin B6', 'Vitamin C', 'Vitamin D', 'Vitamin E', 'Vitamin K', 'Calcium', 'Copper', 'Iron', 
    'Magnesium', 'Manganese', 'Phosphorus', 'Potassium', 'Selenium', 'Zinc'
]

combined_df[per100g_cols] = combined_df[per100g_cols] / 100.0




In [154]:

foods = combined_df.index.tolist()
num_foods = len(foods)
days = range(7)  # 7-day plan

# Extract data into dictionaries
calories = combined_df['Caloric Value'].to_dict()
protein = combined_df['Protein'].to_dict()
fiber = combined_df['Dietary Fiber'].to_dict()
sugars = combined_df['Sugars'].to_dict()
sat_fat = combined_df['Saturated Fats'].to_dict()
vit_c = combined_df['Vitamin C'].to_dict()
iron = combined_df['Iron'].to_dict()
calcium = combined_df['Calcium'].to_dict()
sodium = combined_df['Sodium'].to_dict()

# =========================================
# Parameters (Example Values)
# =========================================

daily_cal_min = 1800
daily_cal_max = 3000
daily_protein_min = 120      
daily_fiber_min = 50        
daily_sugar_max = 15         # grams
weekly_satfat_max = 20      # grams total over 7 days
daily_calcium_min = 140   
daily_sodium_max = 100

daily_vit_c_min = 100       
daily_iron_min = 25          


vitamin_candidates = ['Vitamin A', 'Vitamin B1', 'Vitamin B2', 'Vitamin B3', 'Vitamin B5', 'Vitamin B6',
                      'Vitamin B11', 'Vitamin B12', 'Vitamin C', 'Vitamin D', 'Vitamin E', 'Vitamin K']
mineral_candidates = ['Calcium', 'Copper', 'Iron', 'Magnesium', 'Manganese', 'Phosphorus', 'Potassium', 'Selenium', 'Zinc']
cholesterol_col = 'Cholesterol'
nutrition_density_col = 'Nutrition Density'

# Randomly select 3 vitamins and 2 minerals
chosen_vitamins = random.sample(vitamin_candidates, 3)
chosen_minerals = random.sample(mineral_candidates, 2)

# Extract data for chosen vitamins, minerals, and cholesterol
vitamins_data = {v: combined_df[v].to_dict() for v in chosen_vitamins}
minerals_data = {m: combined_df[m].to_dict() for m in chosen_minerals}
cholesterol_data = combined_df[cholesterol_col].to_dict()
nd_data = combined_df[nutrition_density_col].to_dict()



# Random forbidden foods
num_forbidden = 500
forbidden_foods = set(random.sample(foods, num_forbidden))

# Loved foods (e.g., preferred by the individual)
# Here we select a larger list (e.g., 10 foods) as "loved" at random (excluding forbidden)
possible_loved = [f for f in foods if f not in forbidden_foods]
num_loved = min(500, len(possible_loved))  # ensure we don't exceed available foods
loved_foods = set(random.sample(possible_loved, num_loved))

# Threshold T for diversity
T = 1000.0  # grams per week threshold

model = gp.Model("diet_optimizer_continuous")

# =========================================
# Decision Variables
# =========================================

# x[i,d]: quantity (in grams) of food i consumed on day d
x = model.addVars(num_foods, 7, name="x", lb=0, vtype=GRB.CONTINUOUS)

# z[i]: total weekly consumption of food i
z = model.addVars(num_foods, name="z", lb=0, vtype=GRB.CONTINUOUS)

# w[i]: fraction of inclusion towards diversity
w_vars = model.addVars(num_foods, days, name="w", lb=0, vtype=GRB.CONTINUOUS)

# =========================================
# Constraints
# =========================================

# Link z[i] with x[i,d]
for i in foods:
    model.addConstr(z[i] <= gp.quicksum(x[i,d] for d in days), name=f"total_{i}")

# Daily calorie constraints
for d in days:
    model.addConstr(
        gp.quicksum(calories[i] * x[i,d] for i in foods) >= daily_cal_min,
        name=f"min_cal_day_{d}"
    )
    model.addConstr(
        gp.quicksum(calories[i] * x[i,d] for i in foods) <= daily_cal_max,
        name=f"max_cal_day_{d}"
    )

# Daily protein minimum
for d in days:
    model.addConstr(
        gp.quicksum(protein[i] * x[i,d] for i in foods) >= daily_protein_min,
        name=f"prot_min_day_{d}"
    )

# Daily fiber minimum
for d in days:
    model.addConstr(
        gp.quicksum(fiber[i] * x[i,d] for i in foods) >= daily_fiber_min,
        name=f"fiber_min_day_{d}"
    )

# Daily sugar maximum
for d in days:
    model.addConstr(
        gp.quicksum(sugars[i] * x[i,d] for i in foods) <= daily_sugar_max,
        name=f"sugar_max_day_{d}"
    )

# Weekly saturated fat maximum
model.addConstr(
    gp.quicksum(sat_fat[i] * z[i] for i in foods) <= weekly_satfat_max,
    name="satfat_week_max"
)

# Daily Vitamin C minimum
for d in days:
    model.addConstr(
        gp.quicksum(vit_c[i] * x[i,d] for i in foods) >= daily_vit_c_min,
        name=f"vit_c_min_day_{d}"
    )

# Daily Iron minimum
for d in days:
    model.addConstr(
        gp.quicksum(iron[i] * x[i,d] for i in foods) >= daily_iron_min,
        name=f"iron_min_day_{d}"
    )

for d in days:
    # Daily Calcium Minimum
    model.addConstr(
        gp.quicksum(calcium[i] * x[i,d] for i in foods) >= daily_calcium_min,
        name=f"calcium_min_day_{d}"
    )

for d in days:
    # Daily Sodium Maximum
    model.addConstr(
        gp.quicksum(sodium[i] * x[i,d] for i in foods) <= daily_sodium_max,
        name=f"sodium_max_day_{d}"
    )

# Forbidden foods
for i in forbidden_foods:
    for d in days:
        model.addConstr(x[i,d] <= 0, name=f"forbidden_{i}_{d}")

# w[i] constraints for diversity measure
max_daily_limit_food_based = 300 
max_daily_limit_total_gram = 4000 

for i in foods:
    for d in days:
        model.addConstr(w_vars[i,d] <= 1, name=f"w_le_1_{i}")
        model.addConstr(w_vars[i,d] * T <= z[i], name=f"w_le_zT_{i}")

#limiting the daily consumption from a food
for i in foods:
    for d in days:
        model.addConstr(x[i,d] <= max_daily_limit_food_based, name=f"max_weekly_{i}")

for d in days:
    model.addConstr(gp.quicksum(x[i,d] for i in foods )<= max_daily_limit_total_gram, name=f"max_weekly_{i}")

# =========================================
# Objectives
# =========================================

# 1) Primary Objective: Maximize weekly diversity
weekly_diversity_expr = gp.quicksum(w_vars[i,d] for i in foods for d in days)

# 2) Secondary Objective: Nutrient Balance
# Create expressions for the objective components
vitamin_expr = gp.quicksum(vitamins_data[v][i] * z[i] for v in chosen_vitamins for i in foods)
mineral_expr = gp.quicksum(minerals_data[m][i] * z[i] for m in chosen_minerals for i in foods)
cholesterol_expr = gp.quicksum(cholesterol_data[i] * z[i] for i in foods)
nd_expr = gp.quicksum(nd_data[i] * z[i] for i in foods)

# Define weights
# Give nutrition density a higher weight to prioritize it
w_nd = 10.0   # Higher priority weight for nutrition density
w_vit = 1.0   # Weight for vitamins
w_min = 1.0   # Weight for minerals
w_chol = 1000.0  # Weight for cholesterol

# Construct the final objective
# We want to maximize: (w_nd * ND) + (w_vit * Vitamins) + (w_min * Minerals) - (w_chol * Cholesterol)
nutrient_objective_expr = (w_nd * nd_expr) + 1000*(w_vit * vitamin_expr) +1000*(w_min * mineral_expr) - 1000*(w_chol * cholesterol_expr)

# 3) Tertiary Objective: Maximize consumption of loved foods
# We simply maximize the total amount of these loved foods consumed.
loved_expr = gp.quicksum(z[i] for i in loved_foods)

# Set multi-objectives
# Priority 2: Diversity
model.setObjectiveN(weekly_diversity_expr, index=0, priority=3, name="WeeklyDiversity")

# Priority 1: Nutrient Balance
model.setObjectiveN(nutrient_objective_expr, index=1, priority=2, name="NutrientBalance")

# Priority 3: Loved Foods (least emphasis)
model.setObjectiveN(loved_expr, index=2, priority=1, name="LovedFoods")

# Make sure to set model sense to maximize (for all objectives)
model.ModelSense = GRB.MAXIMIZE

# =========================================
# Solve Model
# ========================================= 
model.reset()
model.optimize()

if model.status == GRB.OPTIMAL:
    print("Optimal solution found.")

    # Extract the solution values for decision variables
    solution_x = model.getAttr('X', x)  # x[i,d].X for each food i and day d


    # Print a clear day-by-day diet plan
    for d_idx, d in enumerate(days):
        print(f"\nDay {d+1} Diet Plan:")
        # Track if any food is selected this day
        foods_selected_today = False
        for i in foods:
            amount = solution_x[(i,d)]
            if amount > 1e-9:  # if the chosen amount is significant
                foods_selected_today = True
                print(f"  {i}: {amount:.1f} g")

        if not foods_selected_today:
            print("  No foods selected.")

    # Compute total weekly cost
    # If you have a cost dictionary similar to how nutrients are handled:
    # For demonstration, we assume 'cost' is a dictionary: cost = {food: cost_per_gram, ...}
    # If not already defined, you need to have 'cost' dictionary defined similarly to calories, etc.
    # For example: cost = combined_df['Cost'].to_dict() 
    # (Make sure to add the Cost column to your combined_df or have another source.)
    if 'Cost' in combined_df.columns:
        cost = combined_df['Cost'].to_dict()
        total_weekly_cost = sum(cost[i]*solution_x[(i,d)] for i in foods for d in days)
        print(f"\nTotal weekly cost: ${total_weekly_cost:.2f}")
    else:
        print("\nNo cost data provided, skipping cost calculations.")

    # Calculate daily totals for each nutrient of interest
    # Add or remove nutrients as needed
    # The following lists are computed day-by-day
    daily_cal = [sum(calories[i]*solution_x[(i,d)] for i in foods) for d in days]
    daily_prot = [sum(protein[i]*solution_x[(i,d)] for i in foods) for d in days]
    daily_fib = [sum(fiber[i]*solution_x[(i,d)] for i in foods) for d in days]
    daily_sug = [sum(sugars[i]*solution_x[(i,d)] for i in foods) for d in days]
    daily_sat = [sum(sat_fat[i]*solution_x[(i,d)] for i in foods) for d in days]
    daily_vit_c_tot = [sum(vit_c[i]*solution_x[(i,d)] for i in foods) for d in days]
    daily_iron_tot = [sum(iron[i]*solution_x[(i,d)] for i in foods) for d in days]
    daily_calcium_tot = [sum(calcium[i]*solution_x[(i,d)] for i in foods) for d in days]
    daily_sodium_tot = [sum(sodium[i]*solution_x[(i,d)] for i in foods) for d in days]

    # Print daily stats
    for d_idx, d in enumerate(days):
        print(f"\nDay {d+1} Stats:")
        print(f"  Calories:    {daily_cal[d_idx]:.1f} kcal")
        print(f"  Protein:     {daily_prot[d_idx]:.1f} g")
        print(f"  Fiber:       {daily_fib[d_idx]:.1f} g")
        print(f"  Sugar:       {daily_sug[d_idx]:.1f} g")
        print(f"  Sat. Fat:    {daily_sat[d_idx]:.1f} g")
        print(f"  Vitamin C:   {daily_vit_c_tot[d_idx]:.4f} mg") 
        print(f"  Iron:        {daily_iron_tot[d_idx]:.4f} mg")
        print(f"  Sodium:      {daily_sodium_tot[d_idx]:.4f} mg")
        print(f"  Calcium:     {daily_calcium_tot[d_idx]:.4f} mg")

    # Compute and print weekly totals for these nutrients if desired
    weekly_cal = sum(daily_cal)
    weekly_prot = sum(daily_prot)
    weekly_fib = sum(daily_fib)
    weekly_sug = sum(daily_sug)
    weekly_sat = sum(daily_sat)
    weekly_vit_c_tot = sum(daily_vit_c_tot)
    weekly_iron_tot = sum(daily_iron_tot)
    weekly_sodium_tot = sum(daily_sodium_tot)
    weekly_calcium_tot = sum(daily_calcium_tot)

    print("\nWeekly Totals:")
    print(f"  Calories:    {weekly_cal:.1f} kcal")
    print(f"  Protein:     {weekly_prot:.1f} g")
    print(f"  Fiber:       {weekly_fib:.1f} g")
    print(f"  Sugar:       {weekly_sug:.1f} g")
    print(f"  Sat. Fat:    {weekly_sat:.1f} g")
    print(f"  Vitamin C:   {weekly_vit_c_tot:.4f} mg")
    print(f"  Iron:        {weekly_iron_tot:.4f} mg")
    print(f"  Sodium:        {weekly_sodium_tot:.4f} mg")
    print(f"  Calcium:        {weekly_calcium_tot:.4f} mg")


    solution_w = model.getAttr('X', w_vars)
    diversity_score = sum(solution_w[i,d] for d in days for i in foods)
    print(f"Diversity Score: {diversity_score}")
    loved_consumption = sum(solution_x[(i,d)] for i in loved_foods for d in days)
    print(f"Total loved foods consumption: {loved_consumption} g")

else:
    print("No optimal solution found. Status:", model.status)
    
    



Discarded solution information
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i5-9300HF CPU @ 2.40GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 57141 rows, 36525 columns and 235067 nonzeros
Model fingerprint: 0xf3aa4467
Variable types: 36525 continuous, 0 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e-05, 1e+03]
  Objective range  [5e-01, 1e+08]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 4e+03]

---------------------------------------------------------------------------
Multi-objectives: starting optimization with 3 objectives ... 
---------------------------------------------------------------------------

Multi-objectives: applying initial presolve ...
---------------------------------------------------------------------------

Presolve removed 41590 rows and 7500 columns
Presolved: 15551 rows, 

UNBOUNDED ANALYSIS

In [ ]:
foods = combined_df.index.tolist()
num_foods = len(foods)
days = range(7)  # 7-day plan

# Extract data into dictionaries
calories = combined_df['Caloric Value'].to_dict()
protein = combined_df['Protein'].to_dict()
fiber = combined_df['Dietary Fiber'].to_dict()
sugars = combined_df['Sugars'].to_dict()
sat_fat = combined_df['Saturated Fats'].to_dict()
vit_c = combined_df['Vitamin C'].to_dict()
iron = combined_df['Iron'].to_dict()

# =========================================
# Parameters (Example Values)
# =========================================

daily_cal_min = 0
daily_cal_max = 52505
daily_protein_min = 50      
daily_fiber_min = 25        
daily_sugar_max = 50         # grams
weekly_satfat_max = 140      # grams total over 7 days

daily_vit_c_min = 0.0075       
daily_iron_min = 0.008          


vitamin_candidates = ['Vitamin A', 'Vitamin B1', 'Vitamin B2', 'Vitamin B3', 'Vitamin B5', 'Vitamin B6',
                      'Vitamin B11', 'Vitamin B12', 'Vitamin C', 'Vitamin D', 'Vitamin E', 'Vitamin K']
mineral_candidates = ['Calcium', 'Copper', 'Iron', 'Magnesium', 'Manganese', 'Phosphorus', 'Potassium', 'Selenium', 'Zinc']
cholesterol_col = 'Cholesterol'
nutrition_density_col = 'Nutrition Density'

# Randomly select 3 vitamins and 2 minerals
chosen_vitamins = random.sample(vitamin_candidates, 3)
chosen_minerals = random.sample(mineral_candidates, 2)

# Extract data for chosen vitamins, minerals, and cholesterol
vitamins_data = {v: combined_df[v].to_dict() for v in chosen_vitamins}
minerals_data = {m: combined_df[m].to_dict() for m in chosen_minerals}
cholesterol_data = combined_df[cholesterol_col].to_dict()
nd_data = combined_df[nutrition_density_col].to_dict()



# Random forbidden foods
num_forbidden = 5
forbidden_foods = set(random.sample(foods, num_forbidden))

# Loved foods (e.g., preferred by the individual)
# Here we select a larger list (e.g., 10 foods) as "loved" at random (excluding forbidden)
possible_loved = [f for f in foods if f not in forbidden_foods]
num_loved = min(20, len(possible_loved))  # ensure we don't exceed available foods
loved_foods = set(random.sample(possible_loved, num_loved))

# Threshold T for diversity
T = 50.0  # grams per week threshold

model = gp.Model("diet_optimizer_continuous")

# =========================================
# Decision Variables
# =========================================

# x[i,d]: quantity (in grams) of food i consumed on day d
x = model.addVars(num_foods, 7, name="x", lb=0, vtype=GRB.CONTINUOUS)

# Daily calorie constraints
for d in days:
    model.addConstr(
        gp.quicksum(calories[i] * x[i,d] for i in foods) >= daily_cal_min,
        name=f"min_cal_day_{d}"
    )
    model.addConstr(
        gp.quicksum(calories[i] * x[i,d] for i in foods) <= daily_cal_max,
        name=f"max_cal_day_{d}"
    )

# Daily protein minimum
for d in days:
    model.addConstr(
        gp.quicksum(protein[i] * x[i,d] for i in foods) >= daily_protein_min,
        name=f"prot_min_day_{d}"
    )

# Daily fiber minimum
for d in days:
    model.addConstr(
        gp.quicksum(fiber[i] * x[i,d] for i in foods) >= daily_fiber_min,
        name=f"fiber_min_day_{d}"
    )

# Daily sugar maximum
for d in days:
    model.addConstr(
        gp.quicksum(sugars[i] * x[i,d] for i in foods) <= daily_sugar_max,
        name=f"sugar_max_day_{d}"
    )

# Weekly saturated fat maximum
model.addConstr(
    gp.quicksum(sat_fat[i] * x[i,d] for d in days  for i in foods) <= weekly_satfat_max,
    name="satfat_week_max"
)

# Daily Vitamin C minimum
for d in days:
    model.addConstr(
        gp.quicksum(vit_c[i] * x[i,d] for i in foods) >= daily_vit_c_min,
        name=f"vit_c_min_day_{d}"
    )

# Daily Iron minimum
for d in days:
    model.addConstr(
        gp.quicksum(iron[i] * x[i,d] for i in foods) >= daily_iron_min,
        name=f"iron_min_day_{d}"
    )

# Forbidden foods
for i in forbidden_foods:
    for d in days:
        model.addConstr(x[i,d] <= 0, name=f"forbidden_{i}_{d}")


    
# =========================================
# Objectives
# =========================================


# 2) Secondary Objective: Nutrient Balance
# Create expressions for the objective components
vitamin_expr = gp.quicksum(vitamins_data[v][i] *x[i,d] for d in days for v in chosen_vitamins for i in foods)
mineral_expr = gp.quicksum(minerals_data[m][i] *x[i,d] for d in days for m in chosen_minerals for i in foods)
cholesterol_expr = gp.quicksum(cholesterol_data[i] * x[i,d] for d in days for i in foods)
nd_expr = gp.quicksum(nd_data[i] * x[i,d] for d in days for i in foods)

# Define weights
# Give nutrition density a higher weight to prioritize it
w_nd = 10.0   # Higher priority weight for nutrition density
w_vit = 1.0   # Weight for vitamins
w_min = 1.0   # Weight for minerals
w_chol = 1.0  # Weight for cholesterol

# Construct the final objective
# We want to maximize: (w_nd * ND) + (w_vit * Vitamins) + (w_min * Minerals) - (w_chol * Cholesterol)
nutrient_objective_expr = (w_nd * nd_expr) + (w_vit * vitamin_expr) + (w_min * mineral_expr) - (w_chol * cholesterol_expr)




# Priority 1: Nutrient Balance
model.setObjective(nutrient_objective_expr)


# Make sure to set model sense to maximize (for all objectives)
model.ModelSense = GRB.MAXIMIZE

# =========================================
# Solve Model
# =========================================
model.reset()
model.Params.DualReductions = 0 
model.Params.InfUnbdInfo = 1 
model.optimize()

if model.status == GRB.OPTIMAL:
    print("Optimal solution found.")
    solution_x = model.getAttr('X', x)
    solution_w = model.getAttr('X', w_vars)
    print("Approximate diversity:", sum(solution_w[i] for i in foods))
    loved_consumption = sum(solution_x[(i,d)] for i in loved_foods for d in days)
    print("Total consumption of loved foods:", loved_consumption)
else:
    print("No optimal solution found. Status:", model.status)


Discarded solution information
Set parameter DualReductions to value 0
Set parameter InfUnbdInfo to value 1
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i5-9300HF CPU @ 2.40GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 85 rows, 17045 columns and 109452 nonzeros
Model fingerprint: 0x215012e5
Coefficient statistics:
  Matrix range     [1e-03, 6e+03]
  Objective range  [2e-02, 4e+04]
  Bounds range     [0e+00, 0e+00]
  RHS range        [7e-03, 5e+04]
Presolve time: 0.03s
Presolved: 43 rows, 17010 columns, 92358 nonzeros

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Ordering time: 0.00s

Barrier performed 0 iterations in 0.06 seconds (0.02 work units)
Barrier solve interrupted - model solved by another algorithm


Solved with dual simplex
Concurrent LP optimizer: dual simplex and barrier
Showing barrie

In [ ]:
if model.status == GRB.UNBOUNDED:
    # Loop through all variables
    for var in model.getVars():
        # Fetch the UnbdRay attribute
        ray_val = var.UnbdRay
        # Check if it's not approximately zero
        if abs(ray_val) > 1e-12:
            print(f"Variable {var.VarName} has UnbdRay = {ray_val}")

Variable x[628,0] has UnbdRay = 1.0
Variable x[628,1] has UnbdRay = 1.0
Variable x[628,2] has UnbdRay = 1.0
Variable x[628,3] has UnbdRay = 1.0
Variable x[628,4] has UnbdRay = 1.0
Variable x[628,5] has UnbdRay = 1.0
Variable x[628,6] has UnbdRay = 1.0
Variable x[1025,0] has UnbdRay = 512.0
Variable x[1025,1] has UnbdRay = 512.0
Variable x[1025,2] has UnbdRay = 512.0
Variable x[1025,3] has UnbdRay = 512.0
Variable x[1025,4] has UnbdRay = 512.0
Variable x[1025,5] has UnbdRay = 512.0
Variable x[1025,6] has UnbdRay = 512.0
Variable x[1047,0] has UnbdRay = 8192.0
Variable x[1047,1] has UnbdRay = 8192.0
Variable x[1047,2] has UnbdRay = 8192.0
Variable x[1047,3] has UnbdRay = 8192.0
Variable x[1047,4] has UnbdRay = 8192.0
Variable x[1047,5] has UnbdRay = 8192.0
Variable x[1047,6] has UnbdRay = 8192.0
Variable x[1505,0] has UnbdRay = 128.0
Variable x[1505,1] has UnbdRay = 128.0
Variable x[1505,2] has UnbdRay = 128.0
Variable x[1505,3] has UnbdRay = 128.0
Variable x[1505,4] has UnbdRay = 128.0
